# 1. Import library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pylab

from scipy import stats
stats.chiqprob = lambda chisq, df: stats.chi2.sf(chisq,df)

# Dataframe manipulation
import numpy as np
import pandas as pd
import math

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# sns.set(style = 'whitegrid',context = 'notebook')

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

# Modelling Helpers:
# from sklearn.preprocessing import Imputer, Normalizer, scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, ShuffleSplit, cross_validate
from sklearn import model_selection
from sklearn.model_selection import train_test_split

# Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
# Evaluation metrics for Classification
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score

# Regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV,ElasticNet,LogisticRegression
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
# Evaluation metrics for Regression 
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score, roc_auc_score, auc,
                             precision_score, recall_score, roc_curve, precision_recall_curve,
                             precision_recall_fscore_support, f1_score,
                             precision_recall_fscore_support)

# NLP
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import string
import nltk
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from collections import Counter
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
import string
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity


# Configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

print("Setup complete...")

Setup complete...


# 2. Load data

In [2]:
df = pd.read_csv('cleaned_ds.csv')

In [3]:
df.shape

(396030, 27)

In [4]:
org_df = df.copy()
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,home_ownership,annual_inc,verification_status,issue_d,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address,Class,num_emp_length
0,10000.0,36,11.44,329.48,B,B4,Marketing,RENT,117000.0,Not Verified,2015-01-01,vacation,Vacation,26.24,1990-06-01,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\nMendozaberg, OK 22690",0,10.0
1,8000.0,36,11.99,265.68,B,B5,Credit analyst,MORTGAGE,65000.0,Not Verified,2015-01-01,debt_consolidation,Debt consolidation,22.05,2004-07-01,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\nLoganmouth, SD 05113",0,4.0
2,15600.0,36,10.49,506.97,B,B3,Statistician,RENT,43057.0,Source Verified,2015-01-01,credit_card,Credit card refinancing,12.79,2007-08-01,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\nNew Sabrina, WV 05113",0,0.5
3,7200.0,36,6.49,220.65,A,A2,Client Advocate,RENT,54000.0,Not Verified,2014-11-01,credit_card,Credit card refinancing,2.60,2006-09-01,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\nDelacruzside, MA 00813",0,6.0
4,24375.0,60,17.27,609.33,C,C5,Destiny Management Inc.,MORTGAGE,55000.0,Verified,2013-04-01,credit_card,Credit Card Refinance,33.95,1999-03-01,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\nGreggshire, VA 11650",1,9.0


In [5]:
df = df.sample(frac = 0.16, random_state = 1)

In [6]:
df.shape

(63365, 27)

In [7]:
df['issue_d'] = pd.to_datetime(df.issue_d)
df['earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line)

datetime_vars = ['issue_d','earliest_cr_line']

In [8]:
## target
target = ['Class']
print("\nTarget var: ", target)

text_vars = []

mixed_vars = ['address']

## cat vars
cat_vars = [var for var in df.columns if df[var].dtypes == 'O' if var not in target]
print("\n========================================================================")
print("\nCategorical vars: ", cat_vars)
print('There are {} categorical variables'.format(len(cat_vars)))

one_level = [var for var in cat_vars if len(df[var].unique()) == 1 if var not in mixed_vars+text_vars]
print("\nOne level vars: ", one_level)
print('There are {} One level variables'.format(len(one_level)))

low_levels = [var for var in cat_vars if len(df[var].unique()) <= 30 if var not in mixed_vars+text_vars]
print("\nLow cardinality vars: ", low_levels)
print('There are {} Low cardinality variables'.format(len(low_levels)))

high_levels = [var for var in cat_vars if len(df[var].unique()) > 30 if var not in mixed_vars+text_vars]
print("\nHigh cardinality vars: ", high_levels)
print('There are {} High cardinality variables'.format(len(high_levels)))

print("\nMixed type vars: ", mixed_vars)
print('There are {} Mixed type  variables'.format(len(mixed_vars)))

num_vars = [var for var in df.columns if df[var].dtypes != 'O' if var not in target]
print("\n========================================================================")
print("\nNumerical vars: ", num_vars)
print('\nThere are {} Numerical variables'.format(len(num_vars)))

dis_vars = [var for var in num_vars if len(df[var].unique()) < 20]
print("\nDiscrete vars: ", dis_vars)
print('There are {} Discrete variables'.format(len(dis_vars)))

cont_vars = [var for var in num_vars if var not in dis_vars]
print("\nContinous vars: ", cont_vars)
print('There are {} Continous variables'.format(len(cont_vars)))

datetime_vars = [var for var in df.columns if df[var].dtypes == '<M8[ns]']
print("\n========================================================================")
print("\nDateTime vars: ", datetime_vars)
print('There are {} DateTime variables'.format(len(datetime_vars)))

print("\n========================================================================")
print("\nText vars: ", text_vars)
print('There are {} Text variables'.format(len(text_vars)))


Target var:  ['Class']


Categorical vars:  ['grade', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'purpose', 'title', 'initial_list_status', 'application_type', 'address']
There are 10 categorical variables

One level vars:  []
There are 0 One level variables

Low cardinality vars:  ['grade', 'home_ownership', 'verification_status', 'purpose', 'initial_list_status', 'application_type']
There are 6 Low cardinality variables

High cardinality vars:  ['sub_grade', 'emp_title', 'title']
There are 3 High cardinality variables

Mixed type vars:  ['address']
There are 1 Mixed type  variables


Numerical vars:  ['loan_amnt', 'term', 'int_rate', 'installment', 'annual_inc', 'issue_d', 'dti', 'earliest_cr_line', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'mort_acc', 'pub_rec_bankruptcies', 'num_emp_length']

There are 16 Numerical variables

Discrete vars:  ['term', 'pub_rec', 'pub_rec_bankruptcies', 'num_emp_length']
There are 4 Discrete variables

Co

# 3. Pre-processing of mixed type of variables

The variables address contain both numbers and letters => create 2 variables for each extracting the numerical and categorical part.

In [9]:
df['zip_code'] = df['address'].apply(lambda address:address[-5:])
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,home_ownership,annual_inc,verification_status,issue_d,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address,Class,num_emp_length,zip_code
35779,8000.0,36,16.20,282.05,C,C4,Counselor,RENT,50000.0,Verified,2013-10-01,debt_consolidation,Consolidation loan,17.64,2000-11-01,13.0,1.0,9034.0,68.4,30.0,w,INDIVIDUAL,0.0,1.0,Unit 9587 Box 8878\nDPO AA 29597,0,7.0,29597
170886,9000.0,36,14.33,309.05,C,C1,American Portfolios,RENT,40000.0,Not Verified,2012-09-01,credit_card,CC Refinance Loan,15.87,1980-05-01,13.0,0.0,14593.0,71.2,30.0,f,INDIVIDUAL,0.0,0.0,"314 Mercer Route\nWest Kathryn, TX 48052",0,4.0,48052
276394,12000.0,60,12.49,269.92,B,B5,Senior Analyst,MORTGAGE,73000.0,Not Verified,2014-07-01,credit_card,Credit card refinancing,28.63,2001-08-01,11.0,0.0,18633.0,62.1,60.0,w,INDIVIDUAL,2.0,0.0,"5389 Brandi Springs\nAshleymouth, LA 29597",0,4.0,29597
196951,20000.0,36,6.62,614.08,A,A2,QA Manager,MORTGAGE,110000.0,Source Verified,2013-12-01,debt_consolidation,Debt Freedom,15.45,1993-10-01,8.0,0.0,5275.0,38.2,24.0,f,INDIVIDUAL,2.0,0.0,"166 Lopez Key Apt. 864\nSouth Angelaland, CO 2...",0,9.0,22690
68891,13600.0,36,7.29,421.74,A,A4,Caxton Associates,MORTGAGE,115000.0,Source Verified,2011-02-01,home_improvement,Home Improvement Loan,6.84,1996-10-01,9.0,0.0,191.0,0.8,17.0,f,INDIVIDUAL,NaN,0.0,48033 Xavier Squares Suite 893\nSouth Rubenbor...,0,7.0,48052


In [10]:
df = df.drop(['address'], axis =1)
cat_vars = [var for var in cat_vars if var not in ['address']]

# 4. Tailored preprocessing / Feature Generation



# 5. Separate train and test set

In [11]:
# Separate into train and test set

X_train, X_test, y_train, y_test = train_test_split(df, df.Class, test_size=0.2, random_state=0, stratify = df.Class)
X_train.shape, X_test.shape

((50692, 27), (12673, 27))

In [12]:
print(y_train.sum()/y_train.count())
print(y_test.sum()/y_test.count())

0.1986309476840527
0.19861122070543674


In [13]:
miss_val_df = pd.DataFrame(df.isnull().sum(),columns=['ColumnName'])
miss_val_df['Percentage'] = 100 * df.isnull().sum()/len(df)
miss_val_df.sort_values(['Percentage'],ascending = False)

,ColumnName,Percentage
mort_acc,6064,9.569952
emp_title,3602,5.684526
num_emp_length,2909,4.590862
title,283,0.446619
pub_rec_bankruptcies,81,0.127831
revol_util,51,0.080486
loan_amnt,0,0.000000
open_acc,0,0.000000
Class,0,0.000000
application_type,0,0.000000


# 6. Continous vars

### 6.1. Handling missing values

In [14]:
miss_val_df = pd.DataFrame(X_train[cont_vars].isnull().sum(),columns=['Count'])
miss_val_df['Percentage'] = 100 * X_train[cont_vars].isnull().sum()/len(df)
miss_val_df.sort_values(['Percentage'],ascending = False)

,Count,Percentage
mort_acc,4831,7.624083
revol_util,38,0.059970
loan_amnt,0,0.000000
int_rate,0,0.000000
installment,0,0.000000
annual_inc,0,0.000000
issue_d,0,0.000000
dti,0,0.000000
earliest_cr_line,0,0.000000
open_acc,0,0.000000


#### mort_acc

Let's fill in the missing mort_acc values based on their total_acc value. If the mort_acc is missing, then we will fill in that missing value with the mean value corresponding to its total_acc value from the Series we created above. This involves using an .apply() method with two columns.

In [15]:
total_acc_avg = df.groupby('total_acc').mean()['mort_acc']
total_acc_avg[2.0]

0.0

In [16]:
def fill_mort_acc(total_acc,mort_acc):
    '''
    Accepts the total_acc and mort_acc values for the row.
    Checks if the mort_acc is NaN , if so, it returns the avg mort_acc value
    for the corresponding total_acc value for that row.
    
    total_acc_avg here should be a Series or dictionary containing the mapping of the
    groupby averages of mort_acc per total_acc values.
    '''
    if np.isnan(mort_acc):
        return total_acc_avg[total_acc]
    else:
        return mort_acc

In [17]:
X_train['mort_acc'] = X_train.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)
X_test['mort_acc'] = X_test.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)

#### revol_util

In [18]:
def impute_na(X_train, df, variable):
    # make temporary df copy
    temp = df.copy()
    
    # extract random from train set to fill the na
    random_sample = X_train[variable].dropna().sample(temp[variable].isnull().sum(), random_state=0)
    
    # pandas needs to have the same index in order to merge datasets
    random_sample.index = temp[temp[variable].isnull()].index
    temp.loc[temp[variable].isnull(), variable] = random_sample
    return temp[variable]

In [19]:
# 'revol_util'
# add variable indicating missingness
for df in [X_train, X_test]:
    for var in ['revol_util']:
        df[var+'_NA'] = np.where(df[var].isnull(), 1, 0)
    
# replace by random sampling
for df in [X_train, X_test]:
    for var in ['revol_util']:
        df[var] = impute_na(X_train, df, var)

### 6.2. Gaussian transformation

### 6.3. Handling Outliers

## 7. Discrete vars

### 7.1. Handling missing data

In [20]:
miss_val_df = pd.DataFrame(X_train[dis_vars].isnull().sum(),columns=['Count'])
miss_val_df['Percentage'] = 100 * X_train[dis_vars].isnull().sum()/len(df)
miss_val_df.sort_values(['Percentage'],ascending = False)

,Count,Percentage
num_emp_length,2303,18.172493
pub_rec_bankruptcies,69,0.544465
term,0,0.000000
pub_rec,0,0.000000


#### pub_rec_bankruptcies, num_emp_length

In [21]:
# 'pub_rec_bankruptcies','num_emp_length'
# add variable indicating missingness
for df in [X_train, X_test]:
    for var in ['pub_rec_bankruptcies','num_emp_length']:
        df[var+'_NA'] = np.where(df[var].isnull(), 1, 0)
    
# replace by random sampling
for df in [X_train, X_test]:
    for var in ['pub_rec_bankruptcies','num_emp_length']:
        df[var] = impute_na(X_train, df, var)

### 7.2. Handling rare values

In [22]:
X_train[cont_vars + dis_vars].isnull().sum()

loan_amnt               0
int_rate                0
installment             0
annual_inc              0
issue_d                 0
dti                     0
earliest_cr_line        0
open_acc                0
revol_bal               0
revol_util              0
total_acc               0
mort_acc                0
term                    0
pub_rec                 0
pub_rec_bankruptcies    0
num_emp_length          0
dtype: int64

In [23]:
X_test[cont_vars + dis_vars].isnull().sum()

loan_amnt               0
int_rate                0
installment             0
annual_inc              0
issue_d                 0
dti                     0
earliest_cr_line        0
open_acc                0
revol_bal               0
revol_util              0
total_acc               0
mort_acc                0
term                    0
pub_rec                 0
pub_rec_bankruptcies    0
num_emp_length          0
dtype: int64

# 8. Categorical vars

### 8.1. Handling missing data

In [24]:
miss_val_df = pd.DataFrame(X_train[cat_vars].isnull().sum(),columns=['Count'])
miss_val_df['Percentage'] = 100 * X_train[cat_vars].isnull().sum()/len(df)
miss_val_df.sort_values(['Percentage'],ascending = False)

,Count,Percentage
emp_title,2858,22.551882
title,238,1.878008
grade,0,0.000000
sub_grade,0,0.000000
home_ownership,0,0.000000
verification_status,0,0.000000
purpose,0,0.000000
initial_list_status,0,0.000000
application_type,0,0.000000


#### emp_title, title

In [25]:
# add label indicating 'Missing' to Cabin categorical
# or replace by most frequent label in Embarked

for df in [X_train, X_test]:
    df['emp_title'].fillna('Missing', inplace=True)  
    df['title'].fillna('Missing', inplace=True)

In [26]:
# check absence of null values
X_train.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
grade                      0
sub_grade                  0
emp_title                  0
home_ownership             0
annual_inc                 0
verification_status        0
issue_d                    0
purpose                    0
title                      0
dti                        0
earliest_cr_line           0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                 0
total_acc                  0
initial_list_status        0
application_type           0
mort_acc                   0
pub_rec_bankruptcies       0
Class                      0
num_emp_length             0
zip_code                   0
revol_util_NA              0
pub_rec_bankruptcies_NA    0
num_emp_length_NA          0
dtype: int64

In [27]:
X_test.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
grade                      0
sub_grade                  0
emp_title                  0
home_ownership             0
annual_inc                 0
verification_status        0
issue_d                    0
purpose                    0
title                      0
dti                        0
earliest_cr_line           0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                 0
total_acc                  0
initial_list_status        0
application_type           0
mort_acc                   0
pub_rec_bankruptcies       0
Class                      0
num_emp_length             0
zip_code                   0
revol_util_NA              0
pub_rec_bankruptcies_NA    0
num_emp_length_NA          0
dtype: int64

### 8.2. High cardinality

### 8.3. Rare values

### 8.4. Encode categorical vars

#### sub_grade, emp_title, title

In [28]:
X_train = X_train.drop(['sub_grade', 'emp_title', 'title'],axis=1)
X_test = X_test.drop(['sub_grade', 'emp_title', 'title'],axis=1)

In [29]:
X_train.shape, X_test.shape

((50692, 27), (12673, 27))

In [30]:
cat_vars = [var for var in cat_vars if var not in ['sub_grade', 'emp_title', 'title']]
cat_vars

['grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type']

#### grade, verification_status, application_type, initial_list_status, purpose

In [31]:
selected_vars = ['grade','verification_status', 'application_type','initial_list_status','purpose']

for var in selected_vars:
    print('Train data',np.sort(X_train[var].unique()))
    print('Test data',np.sort(X_test[var].unique()))

Train data ['A' 'B' 'C' 'D' 'E' 'F' 'G']
Test data ['A' 'B' 'C' 'D' 'E' 'F' 'G']
Train data ['Not Verified' 'Source Verified' 'Verified']
Test data ['Not Verified' 'Source Verified' 'Verified']
Train data ['DIRECT_PAY' 'INDIVIDUAL' 'JOINT']
Test data ['DIRECT_PAY' 'INDIVIDUAL' 'JOINT']
Train data ['f' 'w']
Test data ['f' 'w']
Train data ['car' 'credit_card' 'debt_consolidation' 'educational' 'home_improvement'
 'house' 'major_purchase' 'medical' 'moving' 'other' 'renewable_energy'
 'small_business' 'vacation' 'wedding']
Test data ['car' 'credit_card' 'debt_consolidation' 'educational' 'home_improvement'
 'house' 'major_purchase' 'medical' 'moving' 'other' 'renewable_energy'
 'small_business' 'vacation' 'wedding']


In [32]:
train_dummies = pd.get_dummies(X_train[selected_vars],drop_first=True)
test_dummies = pd.get_dummies(X_test[selected_vars],drop_first=True)

X_train = pd.concat([X_train,train_dummies],axis=1)
X_test = pd.concat([X_test,test_dummies],axis=1)
X_train.drop(selected_vars,axis=1)
X_test.drop(selected_vars,axis=1)

X_train.shape, X_test.shape

((50692, 51), (12673, 51))

In [33]:
X_train = X_train.drop(selected_vars,axis=1)
X_test = X_test.drop(selected_vars,axis=1)

X_train.shape, X_test.shape

((50692, 46), (12673, 46))

#### home_ownership

In [34]:
X_train['home_ownership'].value_counts()

MORTGAGE    25437
RENT        20357
OWN          4883
OTHER          11
NONE            4
Name: home_ownership, dtype: int64

In [35]:
X_train['home_ownership']=X_train['home_ownership'].replace(['NONE', 'ANY'], 'OTHER')
X_test['home_ownership']=X_test['home_ownership'].replace(['NONE', 'ANY'], 'OTHER')

train_dummies = pd.get_dummies(X_train['home_ownership'],drop_first=True)
test_dummies = pd.get_dummies(X_test['home_ownership'],drop_first=True)
X_train = X_train.drop('home_ownership',axis=1)
X_test = X_test.drop('home_ownership',axis=1)
X_train = pd.concat([X_train,train_dummies],axis=1)
X_test = pd.concat([X_test,test_dummies],axis=1)

In [36]:
X_train.shape, X_test.shape

((50692, 48), (12673, 48))

# 9. DateTime vars

### 9.1. Extract data from vars

In [37]:
miss_val_df = pd.DataFrame(X_train[datetime_vars].isnull().sum(),columns=['Count'])
miss_val_df['Percentage'] = 100 * X_train[datetime_vars].isnull().sum()/len(df)
miss_val_df.sort_values(['Percentage'],ascending = False)

,Count,Percentage
issue_d,0,0.0
earliest_cr_line,0,0.0


In [38]:
X_train = X_train.drop(['issue_d','earliest_cr_line'],axis=1)
X_test = X_test.drop(['issue_d','earliest_cr_line'],axis=1)

# 10. Text vars

# 11. Scaling data

In [39]:
X_train.head()

,loan_amnt,term,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies,Class,num_emp_length,zip_code,revol_util_NA,pub_rec_bankruptcies_NA,num_emp_length_NA,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,verification_status_Source Verified,verification_status_Verified,application_type_INDIVIDUAL,application_type_JOINT,initial_list_status_w,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,OTHER,OWN,RENT
107557,5000.0,36,13.67,170.09,40000.0,7.92,5.0,0.0,8098.0,71.0,5.0,0.0,0.0,0,0.5,48052,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
61169,3350.0,36,13.99,114.48,26000.0,30.84,9.0,0.0,24854.0,101.0,17.0,2.0,0.0,1,0.5,86630,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
285973,6000.0,36,11.99,199.26,44000.0,15.25,12.0,0.0,3511.0,28.3,21.0,0.0,0.0,0,1.0,05113,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
61396,6000.0,36,16.29,211.81,120000.0,26.54,20.0,0.0,14830.0,81.5,58.0,9.0,0.0,0,10.0,22690,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
123188,9500.0,36,16.29,335.36,52000.0,5.54,5.0,0.0,8611.0,80.5,8.0,0.0,0.0,0,3.0,29597,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [40]:
removed_list = ['address','Class']
selected_list = [e for e in X_train.columns if e not in removed_list]

X_train = X_train[selected_list]
X_test = X_test[selected_list]

In [41]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train, columns = selected_list)

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns = selected_list)


In [42]:
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [43]:
train_df = pd.concat([X_train, y_train], axis = 1)
train_df.to_csv('train.csv',index=False)
test_df = pd.concat([X_test, y_test], axis = 1)
test_df.to_csv('test.csv',index=False)

In [44]:
X_train.shape

(50692, 45)

In [45]:
X_test.describe()

,loan_amnt,term,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies,num_emp_length,zip_code,revol_util_NA,pub_rec_bankruptcies_NA,num_emp_length_NA,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,verification_status_Source Verified,verification_status_Verified,application_type_INDIVIDUAL,application_type_JOINT,initial_list_status_w,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,OTHER,OWN,RENT
count,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000,12673.000000
mean,0.338885,0.235146,0.322428,0.289095,0.013888,0.184318,0.209544,0.013426,0.012348,0.354044,0.206498,0.056185,0.016864,0.571974,0.357419,0.001026,0.000947,0.047818,0.289908,0.266393,0.160104,0.077645,0.030380,0.007260,0.326994,0.346485,0.997712,0.001657,0.390357,0.210526,0.587864,0.000316,0.061311,0.005287,0.023199,0.011521,0.008680,0.053736,0.000789,0.014125,0.005918,0.003866,0.000395,0.097846,0.402036
std,0.213001,0.424107,0.174691,0.177133,0.013765,0.085866,0.095066,0.039444,0.017656,0.159407,0.104257,0.064602,0.050306,0.378246,0.273750,0.032013,0.030758,0.213390,0.453737,0.442090,0.366717,0.267623,0.171636,0.084896,0.469134,0.475869,0.047784,0.040675,0.487850,0.407699,0.492239,0.017764,0.239910,0.072521,0.150541,0.106718,0.092764,0.225505,0.028081,0.118009,0.076704,0.062063,0.019860,0.297118,0.490328
min,0.006369,0.000000,0.000000,-0.000726,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.182166,0.000000,0.190105,0.161152,0.007969,0.119646,0.148148,0.000000,0.004682,0.236066,0.131579,0.000000,0.000000,0.157895,0.116669,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.286624,0.000000,0.312037,0.249244,0.011770,0.179842,0.185185,0.000000,0.008597,0.360656,0.192982,0.031250,0.000000,0.578947,0.309882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.484076,0.000000,0.427347,0.381699,0.016960,0.243980,0.259259,0.000000,0.015215,0.478689,0.263158,0.093750,0.000000,1.000000,0.508564,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,0.961044,0.956569,1.020433,0.727680,0.888889,1.307692,0.751319,0.798033,1.017544,0.750000,0.714286,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00

# 12. Predictive Model

In [47]:
models = []
models.append(('KNN', KNeighborsClassifier()))
#models.append(('SVC', SVC(kernel="linear")))
#models.append(('LSVC', SVC(kernel="rbf")))
models.append(('LR', LogisticRegression()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('GNB', GaussianNB()))
models.append(('RF', RandomForestClassifier()))
models.append(('GB', GradientBoostingClassifier()))
models.append(('LGB',LGBMClassifier()))
models.append(('ADA',AdaBoostClassifier()))
models.append(('QDA',QuadraticDiscriminantAnalysis()))
models.append(('NN',MLPClassifier()))
models.append(('XGB',XGBClassifier()))

In [51]:
# Test options and evaluation metric
seed = 9
scoring = 'accuracy'

In [52]:
# evaluate each model in turn
results = {}
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state = seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results[name] = cv_results
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

KeyboardInterrupt: 

In [ ]:
results_df = pd.DataFrame(results)
results_df

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=results_df)
plt.show()

In [48]:
def ModelEvaluation(models):
    
    for name,model in models:
        print(name)
        model.fit(X_train, y_train)
        
        print('==========================================================')
        print('Train set')
        y_train_pred = model.predict(X_train)
        print('Accuracy: ', accuracy_score(y_train, list(y_train_pred)))
        print('ROC AUC Score: ', roc_auc_score(y_train, list(y_train_pred)))
        cm_df = pd.DataFrame(confusion_matrix(y_train,list(y_train_pred)), index=model.classes_,columns=model.classes_)
        cm_df.index.name = 'True'
        cm_df.columns.name = 'Predicted'
        print('Confusion matrix')
        print(cm_df)
        print(classification_report(y_train, list(y_train_pred)))
  
        print('----------------------------------------------------------')
        print('Test set')
        y_test_pred = model.predict(X_test)
        print('Accuracy: ', accuracy_score(y_test, list(y_test_pred)))
        print('ROC AUC Score: ', roc_auc_score(y_test, list(y_test_pred)))
        cm_df = pd.DataFrame(confusion_matrix(y_test,list(y_test_pred)), index=model.classes_,columns=model.classes_)
        cm_df.index.name = 'True'
        cm_df.columns.name = 'Predicted'
        print('Confusion matrix')
        print(cm_df)
        print(classification_report(y_test, list(y_test_pred)))
        print('==========================================================')
        
ModelEvaluation(models)

KNN
Train set
Accuracy:  0.8542965359425551
ROC AUC Score:  0.6796555970680185
Confusion matrix
Predicted      0     1
True                  
0          39380  1243
1           6143  3926
              precision    recall  f1-score   support

           0       0.87      0.97      0.91     40623
           1       0.76      0.39      0.52     10069

    accuracy                           0.85     50692
   macro avg       0.81      0.68      0.71     50692
weighted avg       0.84      0.85      0.84     50692

----------------------------------------------------------
Test set
Accuracy:  0.8057287145900733
ROC AUC Score:  0.5929557504440462
Confusion matrix
Predicted     0    1
True                
0          9607  549
1          1913  604
              precision    recall  f1-score   support

           0       0.83      0.95      0.89     10156
           1       0.52      0.24      0.33      2517

    accuracy                           0.81     12673
   macro avg       0.68      0.59

Train set
Accuracy:  0.8881085772903022
ROC AUC Score:  0.7273072061545002
Confusion matrix
Predicted      0     1
True                  
0          40383   240
1           5432  4637
              precision    recall  f1-score   support

           0       0.88      0.99      0.93     40623
           1       0.95      0.46      0.62     10069

    accuracy                           0.89     50692
   macro avg       0.92      0.73      0.78     50692
weighted avg       0.90      0.89      0.87     50692

----------------------------------------------------------
Test set
Accuracy:  0.8860569715142429
ROC AUC Score:  0.7212191051225828
Confusion matrix
Predicted      0     1
True                  
0          10102    54
1           1390  1127
              precision    recall  f1-score   support

           0       0.88      0.99      0.93     10156
           1       0.95      0.45      0.61      2517

    accuracy                           0.89     12673
   macro avg       0.92      